# From YOLO.ipynb

## Obtaining baseline results using Yolo models 

### Data

#### Converting Geojson to YOLO

In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"datasets\Planet\labels")
source_dir = fix_path(r"datasets\Planet\jpg_source")
output_dir = fix_path("datasets/Planet/yolo_labels")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm

# Class mapping
class_map = {
    "marine_debris": 0  # Update with additional classes if needed
}

# Normalize bounding box coordinates
def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# Process .npy files
npy_files = glob(os.path.join(labels_dir, "*.npy"))

for npy_file in tqdm(npy_files):
    # Load .npy data
    data = np.load(npy_file, allow_pickle=True).tolist()

    # Get corresponding image dimensions
    base_name = os.path.splitext(os.path.basename(npy_file))[0]
    image_path = os.path.join(source_dir, base_name + ".jpg")
    
    if not os.path.exists(image_path):
        print(f"Image not found for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Create YOLO label file
    yolo_file_path = os.path.join(output_dir, base_name + ".txt")
    with open(yolo_file_path, "w") as yolo_file:
        for entry in data:
            x_min, y_min, x_max, y_max, class_id = entry
            if class_id > 0:
                class_id -= 1

            # Normalize the bounding box coordinates
            x_center, y_center, width, height = normalize_bbox(
                (x_min, y_min, x_max, y_max), img_width, img_height
            )

            # Write to YOLO file
            yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("Conversion complete!")


  0%|          | 0/739 [00:00<?, ?it/s]

100%|██████████| 739/739 [00:08<00:00, 86.18it/s]

Conversion complete!


### Splitting Dataset

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
labels_dir = "datasets/Planet/yolo_labels"
images_dir = "datasets/Planet/jpg_source"
output_dir = "datasets/Planet/dataset_splits"

# Ratios
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.1

# Ensure the ratios sum to 1
assert round(train_ratio + val_ratio + test_ratio, 5) == 1.0, "Ratios must sum to 1."

# Get list of label files
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Split into train, validation, and test
train_files, temp_files = train_test_split(label_files, test_size=(1 - train_ratio), random_state=42, shuffle=True)
val_files, test_files = train_test_split(temp_files, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42, shuffle=True)

# Function to move files
def move_files(files, dest_images, dest_labels):
    os.makedirs(dest_images, exist_ok=True)
    os.makedirs(dest_labels, exist_ok=True)
    for label_file in files:
        # Move label file
        src_label = os.path.join(labels_dir, label_file)
        dest_label = os.path.join(dest_labels, label_file)
        shutil.copy(src_label, dest_label)

        # Move corresponding image file
        image_file = label_file.replace(".txt", ".jpg")  # Assuming images are .jpg
        src_image = os.path.join(images_dir, image_file)
        dest_image = os.path.join(dest_images, image_file)
        if os.path.exists(src_image):
            shutil.copy(src_image, dest_image)
        else:
            print(f"Image file {image_file} not found for label {label_file}")

# Move files to respective directories
move_files(train_files, os.path.join(output_dir, "train/images"), os.path.join(output_dir, "train/labels"))
move_files(val_files, os.path.join(output_dir, "val/images"), os.path.join(output_dir, "val/labels"))
move_files(test_files, os.path.join(output_dir, "test/images"), os.path.join(output_dir, "test/labels"))

print("Dataset split completed.")
print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")


Dataset split completed.
Train files: 504
Validation files: 101
Test files: 68


#### Augmentation

In [ ]:
import os
import shutil
import random
from glob import glob
from PIL import Image, ImageOps

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"yolo_labels")
source_dir = fix_path(r"NASA_Planet_Data\source")
output_dir = fix_path(r"aug")

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
augmented_images_dir = os.path.join(output_dir, "images")
augmented_labels_dir = os.path.join(output_dir, "labels")
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

# Augmentation functions
def augment_image(image, label_path, base_name, augmentation_type):
    augmented_image = image.copy()
    if augmentation_type == "flip":
        augmented_image = ImageOps.mirror(image)
    elif augmentation_type == "rotate":
        augmented_image = image.rotate(90, expand=True)
    
    # Save augmented image
    augmented_image_path = os.path.join(augmented_images_dir, f"{base_name}_{augmentation_type}.jpg")
    augmented_image.save(augmented_image_path)

    # Copy corresponding label file
    if os.path.exists(label_path):
        augmented_label_path = os.path.join(augmented_labels_dir, f"{base_name}_{augmentation_type}.txt")
        shutil.copy(label_path, augmented_label_path)
    else:
        print(f"Label file {label_path} missing for image {base_name}, skipping...")

# Process images and labels
image_files = glob(os.path.join(source_dir, "*.jpg"))
for image_path in image_files:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(labels_dir, f"{base_name}.txt")

    if not os.path.exists(label_path):
        print(f"Label file missing for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        # Save original image and label
        img.save(os.path.join(augmented_images_dir, f"{base_name}.jpg"))
        shutil.copy(label_path, os.path.join(augmented_labels_dir, f"{base_name}.txt"))

        # Apply augmentations
        augment_image(img, label_path, base_name, "flip")
        augment_image(img, label_path, base_name, "rotate")

print("Dataset augmentation completed.")


Dataset augmentation completed.


In [ ]:
import os

# Directories for augmented dataset
augmented_images_dir = r"aug\images"
augmented_labels_dir = r"aug\labels"

# Count the number of image and label files
num_images = len([f for f in os.listdir(augmented_images_dir) if f.endswith(".jpg")])
num_labels = len([f for f in os.listdir(augmented_labels_dir) if f.endswith(".txt")])

# Print the counts
print(f"Number of augmented images: {num_images}")
print(f"Number of augmented labels: {num_labels}")

# Ensure the counts match
if num_images != num_labels:
    print("Warning: The number of images and labels do not match!")
else:
    print("Dataset is consistent: Images and labels match.")


Number of augmented images: 2217
Number of augmented labels: 2217
Dataset is consistent: Images and labels match.


---

# Main Training

In [2]:
# Check pytorch gpu detection
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:
import os
from ultralytics import YOLO

# Load a model
model = YOLO("runs/train/train3/weights/best.pt")  # load a pretrained model (recommended for training)
project = "runs/train"
name = "yolo11m_trained_frozen"
os.makedirs(project, exist_ok=True)

In [7]:
# Train the model
results = model.train(
    data="datasets/Planet/dataset_splits_new/data.yaml",
    epochs=300,
    imgsz=256,
    batch=16,
    device='cuda',
    optimizer="SGD",
    workers=8,
    lr0=0.001,
    patience=30,
    project=project,
    name=name,
    plots=True,
    freeze=22  # 22 layers seem to be part of the feature extraction module, before the detection head. 
)

New https://pypi.org/project/ultralytics/8.3.74 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.73  Python-3.10.16 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=runs/train/train3/weights/best.pt, data=datasets/Planet/dataset_splits_new/data.yaml, epochs=300, time=None, patience=30, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs/train, name=yolo11m_trained_frozen, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=22, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

train: Scanning C:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_new\train\labels.cache... 2868 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2868/2868 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_new\val\labels.cache... 96 images, 0 backgrounds, 0 corrupt: 100%|██████████| 96/96 [00:00<?, ?it/s]


Plotting labels to runs\train\yolo11m_trained_frozen\labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\train\yolo11m_trained_frozen
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     73/300     0.218G      1.353     0.9829      1.073         11        256: 100%|██████████| 180/180 [00:27<00:00,  6.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.83it/s]

                   all         96        330      0.745      0.694      0.741      0.412
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 43, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



73 epochs completed in 0.654 hours.
Optimizer stripped from runs\train\yolo11m_trained_frozen\weights\last.pt, 5.4MB
Optimizer stripped from runs\train\yolo11m_trained_frozen\weights\best.pt, 5.4MB

Validating runs\train\yolo11m_trained_frozen\weights\best.pt...
Ultralytics 8.3.73  Python-3.10.16 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.89it/s]


                   all         96        330      0.718      0.726      0.758       0.42
Speed: 0.1ms preprocess, 2.7ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\train\yolo11m_trained_frozen


In [3]:
import os
from glob import glob
from ultralytics import YOLO

images = list(sorted(glob("datasets/Planet/dataset_splits_new/val/images/*.jpg")))
output_dir = "inference_output"
os.makedirs(output_dir, exist_ok=True)

# Load a model
model = YOLO("runs/train/yolo11m2/weights/best.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(
    images,
    iou=0.5,
    imgsz=256,
    device='cuda',
    project="runs/detect",
    name="val_set",
    save=True,
    save_txt=True,
    show_labels=False,
    show_conf=True
)  # return a list of Results objects

# Process results list
# for i, result in enumerate(results):
    # boxes = result.boxes  # Boxes object for bounding box outputs
    # masks = result.masks  # Masks object for segmentation masks outputs
    # keypoints = result.keypoints  # Keypoints object for pose outputs
    # probs = result.probs  # Probs object for classification outputs
    # obb = result.obb  # Oriented boxes object for OBB outputs
    # # result.show()  # display to screen
    # result.save(filename=os.path.join(output_dir, str(i).zfill(4) + ".jpg"))  # save to disk


0: 256x256 1 debris, 5.1ms
1: 256x256 4 debriss, 5.1ms
2: 256x256 1 debris, 5.1ms
3: 256x256 1 debris, 5.1ms
4: 256x256 1 debris, 5.1ms
5: 256x256 (no detections), 5.1ms
6: 256x256 1 debris, 5.1ms
7: 256x256 2 debriss, 5.1ms
8: 256x256 1 debris, 5.1ms
9: 256x256 3 debriss, 5.1ms
10: 256x256 2 debriss, 5.1ms
11: 256x256 (no detections), 5.1ms
12: 256x256 4 debriss, 5.1ms
13: 256x256 4 debriss, 5.1ms
14: 256x256 1 debris, 5.1ms
15: 256x256 3 debriss, 5.1ms
16: 256x256 2 debriss, 5.1ms
17: 256x256 1 debris, 5.1ms
18: 256x256 1 debris, 5.1ms
19: 256x256 2 debriss, 5.1ms
20: 256x256 6 debriss, 5.1ms
21: 256x256 5 debriss, 5.1ms
22: 256x256 6 debriss, 5.1ms
23: 256x256 6 debriss, 5.1ms
24: 256x256 1 debris, 5.1ms
25: 256x256 (no detections), 5.1ms
26: 256x256 6 debriss, 5.1ms
27: 256x256 1 debris, 5.1ms
28: 256x256 2 debriss, 5.1ms
29: 256x256 1 debris, 5.1ms
30: 256x256 4 debriss, 5.1ms
31: 256x256 5 debriss, 5.1ms
32: 256x256 5 debriss, 5.1ms
33: 256x256 4 debriss, 5.1ms
34: 256x256 2 deb